In [1]:
import tensorflow as tf
import numpy as np
import scipy.linalg as ln
from model.ntm_ops import *
from model.memory import *
%load_ext autoreload
%autoreload 2

In [2]:
sess = tf.InteractiveSession()

# Ops Testing

In [6]:
hidden_state = tf.constant(np.array([[1., 2., 3.]]), dtype=tf.float32)
cell_state = tf.constant(np.array([[2, 3, 4.]]), dtype=tf.float32)
read_vector = tf.constant(np.array([[0.5, 0.1]]), dtype=tf.float32)
res = linear([hidden_state, cell_state, read_vector], bias=True, bias_start=0.0, output_size=4*3,scope="test")
print "shape of result", res.get_shape().as_list()
sess.run(tf.initialize_all_variables())
print "value of result", res.eval()

shape of result [1, 12]
value of result [[ 0.84255314  2.23059011  4.0013032   0.38918835  1.01767564 -0.33680916
   1.1560061  -0.30750984 -1.28430939 -0.20256931  0.32671487 -4.23408842]]


## Introduction
Neural Turing Machines combined the ability of Turing Machine and Neural Networks to infer simple algorithms. The controller (it's usually a LSTM) can be viewed as CPU and the external memory can be seen as RAM. 

A NTM has four components: Controller, read heads, write heads, and an external memory. 

High level overview:
1. Addressing: Addressing mechanism is used to produce the weightings of each head. There are two types of adrressing, content based and location based. At every time step, the controller outputs five elements to produce weightings of each head: key vector, key strength, interpolation gate, shift weighting, and a scalar that used to sharpen the weightings. 
2. Read: each read head has a weighting vector tells how much degree of information we read from on each memory location
3. Write: each write head has a weighting vector, an erase vector and an add vector. This is inspired by LSTM's forget gate and input gate. 

## Section 1 Hyper parameters

### 1.1 Memory matrix
Define two hyper parameters for the memory matrix: $N \times M$, where $N$ is the number of memory locations, $M$ is the vector size at each memory location

### 1.2 Controller dimension
Define the LSTM hidden state dimension h and stacked hidden layer number a. This is the same as tradition LSTM with the hidden state and cell state.

Define the output and input dimension, in NTM, it usually is how many bits per sequence. e.g. If one of the input sequence is [0, 1, 0, 1, 0, 1], then it should be 6.

### 1.3 The range of allowed location shift
Define the range of the allowed location shift in location based addressing (Convolutional shift), s. e.g. if s = 3, then allowed location shift will be [-1, 0, 1]

## Section 2 Memory Testing
At every time step the controller outputs weighting of each head and hidden states(including cell states in original LSTM).. The weighting is determined by addressing mechanism:
1. Content Addressing
2. Interpolation
3. Convolutional Shift
4. Sharpening

In [3]:
# define memory and give attributes values
B, N, M = 1, 3, 2
memory = Memory(batch_size=B, mem_dim=M, mem_size=N)
memory.memory = tf.constant(np.array([[
            [6, 8],
            [1, 2],
            [3, 3],
            
        ]]), dtype=tf.float32)
memory.read_weighting = tf.constant(np.array([
            [1, 0, 0]
        ]), dtype=tf.float32)
memory.write_weighting = tf.constant(np.array([
            [0, 1, 0]
        ]), dtype=tf.float32)
memory.read_vector = tf.constant(np.array([
            [1.0, 1.0]
        ]), dtype=tf.float32)

In [4]:
# define emitted vectors, these should be generated by the controller.
key_vector = tf.constant(np.array([[3, 4]]), dtype=tf.float32)
key_strength = tf.constant(np.array([[1]]), dtype=tf.float32)
interplotation = tf.constant(np.array([[0.9]]), dtype=tf.float32)
shifting = tf.constant([[0, 0, 1]], dtype=tf.float32)
sharpening = tf.constant([[2]], dtype=tf.float32)
add_vector = tf.constant([
        [0.5, 0.5]
    ])
erase_vector = tf.constant([
        [0.1, 0.5]
    ])

In [5]:
# memory write test
print "Previous memory\n", memory.memory.eval()
matrix = memory.write(memory.write_weighting, memory.memory, erase_vector, add_vector)
memory.memory = matrix
print "Current memory\n", memory.memory.eval()

Previous memory
[[[ 6.  8.]
  [ 1.  2.]
  [ 3.  3.]]]
Current memory
[[[ 6.          8.        ]
  [ 1.39999998  1.5       ]
  [ 3.          3.        ]]]


In [6]:
# update write weighting test
print "current write weighting", memory.write_weighting.eval()
memory.write_weighting = memory.update_weighting(key_vector, key_strength, interplotation, shifting, sharpening,
                                                 memory.write_weighting, memory.memory)
print "next write weighting", memory.write_weighting.eval()

current write weighting [[ 0.  1.  0.]]
next write weighting [[ 0.32519153  0.32578066  0.34902781]]


In [7]:
# memory read test
print "current read vector", memory.read_vector.eval()
memory.read_vector = memory.read(memory.read_weighting, memory.memory)
print "next read vector", memory.read_vector.eval()

current read vector [[ 1.  1.]]
next read vector [[ 6.  8.]]


In [8]:
# update read weighting test
print "current read weighting", memory.read_weighting.eval()
memory.read_weighting = memory.update_weighting(key_vector, key_strength, interplotation, shifting, sharpening,
                                                 memory.read_weighting, memory.memory)
print "next read weighting", memory.read_weighting.eval()

current read weighting [[ 1.  0.  0.]]
next read weighting [[ 0.32514414  0.34946215  0.32539377]]


## Section 2 NTM Cell

In [3]:
from model.ntm_cell import *

In [4]:
NTMStateTuple = collections.namedtuple("NTMStateTuple", ("h", "c", "m"))

In [5]:
cell = NTMCell(3, 2, 5)

In [6]:
memory = Memory(mem_dim=5, mem_size=2)

In [16]:
state = NTMStateTuple(tf.constant(np.array([[1, 2, 3]]), name="cell", dtype=tf.float32), 
                      tf.constant(np.array([[2, 5, 6]]), name="hidden", dtype=tf.float32), memory)

In [8]:
inputs = tf.placeholder(name="inputs", dtype=tf.float32, shape=(1, 2))

In [17]:
tf.get_variable_scope().reuse_variables()
out, state = cell(inputs, state)

In [18]:
sess.run(tf.initialize_all_variables())

In [19]:
out.eval(feed_dict={inputs:np.array([[1.0, 2.0]])})

array([[ 0.74140698,  0.27215773,  0.74529713]], dtype=float32)

In [20]:
m = state[2]

In [21]:
m.read_weighting.eval(feed_dict={inputs:np.array([[1.0, 2.0]])})

array([[ 0.5,  0.5]], dtype=float32)

In [22]:
m.write_weighting.eval(feed_dict={inputs:np.array([[1.0, 2.0]])})

array([[ 0.5,  0.5]], dtype=float32)

In [23]:
m.memory.eval(feed_dict={inputs:np.array([[1.0, 2.0]])})

array([[[ 0.22362523,  0.22423317, -0.02385022,  0.36904815, -0.00376798],
        [ 0.22362523,  0.22423317, -0.02385022,  0.36904815, -0.00376798]]], dtype=float32)

In [24]:
m.read_vector.eval(feed_dict={inputs:np.array([[1.0, 2.0]])})

array([[ 0.22362523,  0.22423317, -0.02385022,  0.36904815, -0.00376798]], dtype=float32)

In [27]:
for v in tf.all_variables():
    print v.name

ntm_cell/lstm/linear/Matrix:0
ntm_cell/lstm/linear/bias:0
ntm_cell/addressing/key_vector/Matrix:0
ntm_cell/addressing/key_strength/Matrix:0
ntm_cell/addressing/interpolation/Matrix:0
ntm_cell/addressing/shifting/Matrix:0
ntm_cell/addressing/sharpening/Matrix:0
ntm_cell/write/erase/Matrix:0
ntm_cell/write/add/Matrix:0


In [32]:
a = [n.name for n in tf.get_default_graph().as_graph_def().node]

In [33]:
a

[u'external_memory/memory/dims',
 u'external_memory/memory/value',
 u'external_memory/memory',
 u'external_memory/read_weighting/dims',
 u'external_memory/read_weighting/value',
 u'external_memory/read_weighting',
 u'external_memory/write_weighting/dims',
 u'external_memory/write_weighting/value',
 u'external_memory/write_weighting',
 u'external_memory/read_vector/dims',
 u'external_memory/read_vector/value',
 u'external_memory/read_vector',
 u'cell',
 u'hidden',
 u'inputs',
 u'ntm_cell/lstm/linear/Matrix',
 u'ntm_cell/lstm/linear/Matrix/Initializer/random_uniform/shape',
 u'ntm_cell/lstm/linear/Matrix/Initializer/random_uniform/min',
 u'ntm_cell/lstm/linear/Matrix/Initializer/random_uniform/max',
 u'ntm_cell/lstm/linear/Matrix/Initializer/random_uniform/RandomUniform',
 u'ntm_cell/lstm/linear/Matrix/Initializer/random_uniform/sub',
 u'ntm_cell/lstm/linear/Matrix/Initializer/random_uniform/mul',
 u'ntm_cell/lstm/linear/Matrix/Initializer/random_uniform',
 u'ntm_cell/lstm/linear/Matrix/